In [ ]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>") 

In [ ]:
import numpy as np
from pypower.api import *
from scipy import sparse
import copy
import h5py
import itertools
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
import torch.nn as nn
import time
from itertools import count
import math

from alegnn.utils import graphML
from alegnn.modules import architecturesTime

In [ ]:
def environmentStep(actionSpace, LOADING_FACTOR):

    # make sure to take care of off-by-one errors!
    # ppc = case14()
    # ppc = case30()
    ppc = case39()
    # ppc = case118()
    
    # print(ppc.keys())

    # Checking the sizes before pruning the testcases (confirmation with MATLAB (1-13));
    branches = ppc["branch"]
    #print("branch: ", branches.shape)
    buses = ppc["bus"]
    #print("bus: ", buses.shape)
    generation = ppc["gen"]
    #print("generation: ", generation.shape)
    #print(branches[0,:])
    #print(generation[0,:])
    #print(buses)


    # Making sure that the dimensions are as expected (as in MATLAB);
    # 0:13 in Python equvivalent to 1:13 in MATLAB! 
    ppc["branch"] = ppc["branch"][:, 0:13]
    ppc["bus"] = ppc["bus"][:, 0:13]
    ppc["gen"] = ppc["gen"][:, 0:21]
    #print(ppc["bus"])


    # Varying the Loading Condition;
    ppc["bus"][:,2] = LOADING_FACTOR*ppc["bus"][:,2] # real power demand P
    ppc["bus"][:,3] = LOADING_FACTOR*ppc["bus"][:,3] # reactive power demand Q 
    ppc["gen"][:,1] = LOADING_FACTOR*ppc["gen"][:,1] # real power output P_gen
    ppc["gen"][:,2] = LOADING_FACTOR*ppc["gen"][:,2] # reactive power output Q_gen
    #print(ppc["bus"])


    # Relabeling (for some tests cases the indices do not start from 1) 
    # the Bus Numbers in Increasing Order (Data Pre-Processing);
    bus = ppc["bus"]
    nodeSelectIndex = np.zeros((bus.shape[0], 2))
    for index in range(bus.shape[0]):
        nodeSelectIndex[index, 0] = index + 1      # new bus index 
        nodeSelectIndex[index, 1] = bus[index, 0]  # old bus index 
        bus[index, 0] = index + 1

    bus = bus[bus[:,0].argsort()]
    ppc["bus"] = bus
    #print(ppc["bus"])
    #print(nodeSelectIndex)
    #print(nodeSelectIndex.shape)


    # Accordingly, relabeling the generator buses (Data Pre-Processing);
    gen = ppc["gen"]
    for index in range(gen.shape[0]):
        gen[index, 0] = nodeSelectIndex[np.argwhere(gen[index,0]==nodeSelectIndex[:,1]), 0]
    gen = gen[gen[:,0].argsort()] 
    ppc["gen"] = gen
    #print(ppc["gen"])


    # Since branches are connected to nodes, the "from" and "to" attributes should be relabeled (Data Pre-Processing);
    branch_tmp = ppc["branch"]
    for index in range(branch_tmp.shape[0]):
        # from attribute
        branch_tmp[index, 0] = nodeSelectIndex[np.argwhere(branch_tmp[index,0]==nodeSelectIndex[:,1]), 0]
        # to attribute
        branch_tmp[index, 1] = nodeSelectIndex[np.argwhere(branch_tmp[index,1]==nodeSelectIndex[:,1]), 0]
    ppc["branch"] = branch_tmp
    #print(ppc["branch"])


    # In the case of parallel lines, add their rates/capacities/long-term ratings (Data Pre-Processing);
    # print(ppc["branch"].shape
    # for each branch
    for i in range(ppc["branch"].shape[0]):

        # go through all the other branches
        for j in range(i+1, ppc["branch"].shape[0]):

            if (ppc["branch"][i,0]==ppc["branch"][j,0] and ppc["branch"][i,1]==ppc["branch"][j,1]) or (ppc["branch"][i,0]==ppc["branch"][j,1] and ppc["branch"][i,1]==ppc["branch"][j,0]):

                ppc["branch"][i, 5] = ppc["branch"][i, 5] + ppc["branch"][j, 5]
                ppc["branch"][j, :] = 0

    ppc["branch"] = np.delete(ppc["branch"], np.argwhere(ppc["branch"][:, 0]==0), axis=0)
    branch = ppc["branch"]
    reactanceVector = branch[:,3]


    # Ideally, I'd want that GENERATION = DEMAND; So this ensures that the NET power injection sums to 0; 
    # In some test cases, this might not be. So, the following code ensures that the demand and supply remain balanced!
    # this is done by modifying the load in order to match generation!
    powerInj_consume = bus[:,2]                              # real-power consumption vector
    powerInj_supply = np.zeros((powerInj_consume.shape[0]))  # real-power generation vector

    for index in range(ppc["gen"].shape[0]):
        powerInj_supply[np.argwhere(bus[:,0]==gen[index,0])] = powerInj_supply[np.argwhere(bus[:,0]==gen[index,0])] + gen[index, 1]

    # real NET-power injection vector    
    powerInj_vector = powerInj_supply - powerInj_consume 
    #print(np.sum(powerInj_vector))

    # Increading/Decreasing the loads or consumption vector (load data)!
    powerInj_consume = powerInj_consume + np.sum(powerInj_vector)/powerInj_vector.shape[0]

    # NEW real NET-power injection vector  
    powerInj_vector = powerInj_supply - powerInj_consume
    # print(np.sum(powerInj_vector))

    # Print the current generation (and thus, same load) in the network!
    currentLoad = np.sum(ppc["bus"][:,2])
    threshold = 0.01*currentLoad
    #print(currentLoad, threshold)


    
    # Building the Adjacency, Capacity and Reactance Matrix;
    Adj = np.zeros((bus.shape[0], bus.shape[0]))
    Reactance = np.zeros((bus.shape[0], bus.shape[0]))
    Capacity = np.zeros((bus.shape[0], bus.shape[0]))
    for index in range(branch.shape[0]):
        Adj[ int(branch[index, 0])-1, int(branch[index, 1])-1 ] = 1
        Adj[ int(branch[index, 1])-1, int(branch[index, 0])-1 ] = 1

        Reactance[ int(branch[index, 0])-1, int(branch[index, 1])-1 ] = branch[index, 3]
        Reactance[ int(branch[index, 1])-1, int(branch[index, 0])-1 ] = branch[index, 3]

        if branch[index, 5] > 0:
            Capacity[ int(branch[index,0])-1, int(branch[index,1])-1 ] = branch[index, 5]
            Capacity[ int(branch[index,1])-1, int(branch[index,0])-1 ] = branch[index, 5]
        else:
            Capacity[ int(branch[index,0])-1, int(branch[index,1])-1 ] = 9900
            Capacity[ int(branch[index,1])-1, int(branch[index,0])-1 ] = 9900

    Capacity_vector = branch[:,5]
    Capacity_vector = np.expand_dims(Capacity_vector, axis=1)
    count_of_zero_capacity_branches = 0
    for index in range(Capacity_vector.shape[0]):
        if Capacity_vector[index, 0] == 0:
            count_of_zero_capacity_branches += 1
            Capacity_vector[index, 0] = 9900

    #print("count_of_zero_capacity_branches: ", count_of_zero_capacity_branches)        
    for index in range(branch.shape[0]):
        branch[index,5] = Capacity_vector[index, 0]
    ppc["branch"] = branch
###########----###########----###########----###########----###########----###########----###########----


    def conncomp(link_row, link_column):
        """
        A function to run DFS on a graph constructed from branches end indices.
        link_row : From indices 
        link_column : To indices
        """
        def graph_construct(link_row, link_column):
            graph = {node: [] for node in range(1, 39+1)}
            for a, b in zip(list(link_row), list(link_column)):
                a = a.item()
                b = b.item()
                graph[a].append(b)
                graph[b].append(a)
            return graph

        def explore(graph, current, visited, count, C):
            if current in visited:
                return False 

            visited.add(current)
            C[current-1] = count+1
            for neighbor in graph[current]:
                explore(graph, neighbor, visited, count, C)

            return True    

        graph = graph_construct(link_row, link_column)
        C = np.zeros((39, 1))
        visited = set()
        count = 0

        for node in graph:
            if explore(graph, node, visited, count, C) == True:
                count += 1
        return count, C
###########----###########----###########----###########----###########----###########----###########----


    def b_Pypower_DC_OPA_slack(voltageAngles, ppc, adjacency, capacity, capacity_vector, alpha, initFail, tmp_index, link_flag_vector = np.ones((46, 1))):
        """
        voltageAngles (numpy array)    : (modify, Nx1) array should be filled-up with measurements at each bus
        ppc (dict)                     : (modify)      testcase prior to starting the removal process
        adjacency (sparse matrix)      : (modify, NxN) 
        capacity (numpy array)         : (NxN) maximum capacity of each branch as a 2-d array 
        capacity_vector (numpy array)  : (Lx1) maximum capacity of each branch as 1-d array
        alpha (int)                    : 1 
        initFail ()                    : 
        tmp_index ()                   :
        link_flag_vector (numpy array) : (modify, Lx1) To keep track of all the line outages along the way!
        """
        
        # as of now, adjacency is of type "csr.matrix"!
        result = rundcpf_my(ppc)

        temp = ppc["branch"][:, 13]
        temp = np.expand_dims(temp, axis = 1)
        result[0]["branch"] = np.concatenate([result[0]["branch"], temp], axis = 1)

        start_gen = np.sum(result[0]["gen"][:, 1])
        start_load = np.sum(result[0]["bus"][:, 2])

        tempur_vector = abs(result[0]["branch"][:, 13])                            #  (L, ) 
        tempur = np.zeros((result[0]["bus"].shape[0], result[0]["bus"].shape[0]))  #  (N x N) matrix

        for index in range(tempur_vector.shape[0]):
            tempur[ int(result[0]["branch"][index, 0])-1, int(result[0]["branch"][index, 1])-1 ] = tempur_vector[index]
            tempur[ int(result[0]["branch"][index, 1])-1, int(result[0]["branch"][index, 0])-1 ] = tempur_vector[index]

        for index in range(initFail.shape[0], 1):
            adjacency[ int(initFail[index, 0])-1, int(initFail[index, 1])-1] = 0
            adjacency[ int(initFail[index, 1])-1, int(initFail[index, 0])-1] = 0

        # print(ppc["branch"].shape)                                     # (L x 14)
        # link_flag_vector = np.ones((result[0]["branch"].shape[0], 1))  # (L x  1)  # check again
        link_flag_vector[tmp_index.item(), 0] = 0                                    # check again

        for t in range(tmp_index.shape[0]):
            # action cannot be executed since already taken!
            if np.argwhere(ppc["branch"][:,13] == tmp_index[t]).shape[0]==0:
                print("Branch already removed!")
                gen_shed = start_gen - np.sum(result[0]["gen"][:, 1])
                load_shed = start_load - np.sum(result[0]["bus"][:, 2])
                link_row = ppc["branch"][:, 0]                      # branch FROM
                link_row = np.expand_dims(link_row, axis=1)
                link_column = ppc["branch"][:, 1]                   # branch TO
                link_column = np.expand_dims(link_column, axis=1)

                link_row = link_row.astype(int)
                link_column = link_column.astype(int)
                S, C = conncomp(link_row, link_column)    
                powerFlowVector = result[0]["branch"][:, 13]

                return (powerFlowVector, ppc, adjacency, S, gen_shed, load_shed, link_flag_vector)
            else:    
                ppc["branch"] = np.delete(ppc["branch"], (np.argwhere(ppc["branch"][:,13] == tmp_index[t])), axis=0)


        # print(result[0]["branch"].shape[0])
        # print(np.sum(link_flag_vector))
        # print(ppc["branch"].shape[0])

        link_row = ppc["branch"][:, 0]                      # branch FROM
        link_row = np.expand_dims(link_row, axis=1)

        link_column = ppc["branch"][:, 1]                   # branch TO
        link_column = np.expand_dims(link_column, axis=1)

        # BFS traversal for connected components!
        # print(link_row, link_column)
        link_row = link_row.astype(int)
        link_column = link_column.astype(int)
        S, C = conncomp(link_row, link_column)

        # Load Shedding
        if S > 1:
            result_tmp = copy.deepcopy(result)
            result_tmp[0]["bus"] = np.array([])
            result_tmp[0]["gen"] = np.array([])
            result_tmp[0]["branch"] = np.array([])

            for index in range(1, S+1):
                #print("----------just check----------")
                #print(index)
                #print("----------just check----------") 

                CC_Index = np.argwhere(C == index)      # (-, 2) 2-D indices of numpy array satisfying given condition
                #print(CC_Index[:, 0])
                CC_Load = ppc["bus"][CC_Index[:, 0], 2] # (-,  )
                CC_Gen = np.array([])
                CC_Gen_Index = np.array([])             # (-,  )


                # finding generators and generators indices in this island!
                for j in range(ppc["gen"][:, 0].shape[0]):
                    if np.argwhere(CC_Index[:, 0] == ppc["gen"][j, 0]).shape[0]==1: 
                        CC_Gen = np.append(CC_Gen, ppc["gen"][j, 1])
                        CC_Gen_Index = np.append(CC_Gen_Index, j)

                CC_Gen_Index = CC_Gen_Index.astype(int)


                # controlled load-generation balance in each island!
                if np.sum(CC_Load) < np.sum(CC_Gen):
                    if np.sum(CC_Load) > 0:
                        CC_Gen = CC_Gen*(np.sum(CC_Load)/np.sum(CC_Gen))
                        # CC_Gen = np.reshape(CC_Gen, (-1, 1))
                        ppc["gen"][CC_Gen_Index, 1] = CC_Gen
                    else:                
                        ppc["gen"][CC_Gen_Index, 1] = 0
                        ppc["bus"][CC_Index[:, 0], 2] = 0
                elif np.sum(CC_Load) > np.sum(CC_Gen):
                    islandCapacity = np.sum(ppc["gen"][CC_Gen_Index, 8])
                    if np.sum(CC_Gen)==0:
                        ppc["gen"][CC_Gen_Index, 1] = 0
                        ppc["bus"][CC_Index[:, 0], 2] = 0
                    elif islandCapacity >= np.sum(CC_Load):
                        CC_Gen = CC_Gen + ppc["gen"][CC_Gen_Index, 8]*(np.sum(CC_Load) - np.sum(CC_Gen))/islandCapacity
                        #CC_Gen = np.reshape(CC_Gen, (-1, 1))
                        ppc["gen"][CC_Gen_Index, 1] = CC_Gen
                    else:
                        CC_Gen = ppc["gen"][CC_Gen_Index, 8]
                        #CC_Gen = np.reshape(CC_Gen, (-1, 1))
                        ppc["gen"][CC_Gen_Index, 1] = CC_Gen

                        CC_Load = CC_Load*(np.sum(CC_Gen)/np.sum(CC_Load))
                        #CC_Load = np.reshape(CC_Load, (-1, 1))
                        ppc["bus"][CC_Index[:, 0], 2] = CC_Load

                # print(np.sum(CC_Load), np.sum(CC_Gen))

                ppc_island = copy.deepcopy(ppc)
                ppc_island["bus"] = ppc["bus"][CC_Index[:, 0], :]
                ppc_island["gen"] = ppc["gen"][CC_Gen_Index, :]
                ppc_island["gencost"] = ppc["gencost"][CC_Gen_Index, :]


                # assigning slack bus to the island if original not present to successfully solve power-flow! 
                if np.argwhere(ppc_island["bus"][:, 1]==3).shape[0]==0 and ppc_island["bus"][:, 1].shape[0]>1:
                    island_slack_set = np.argwhere(ppc_island["bus"][:, 2] == min(ppc_island["bus"][:, 2]))
                    island_slack = island_slack_set[0].item()
                    ppc_island["bus"][island_slack, 1] = 3

                # find all branches that belong to this island!
                CC_Branch_Index = np.array([])
                for t in range(ppc["branch"].shape[0]):
                    if np.argwhere(ppc_island["bus"][:, 0]==ppc["branch"][t, 0]).shape[0]>0 and np.argwhere(ppc_island["bus"][:, 0]==ppc["branch"][t, 1]).shape[0]>0: 
                        CC_Branch_Index = np.append(CC_Branch_Index, t)

                CC_Branch_Index = CC_Branch_Index.astype(int)
                ppc_island["branch"] = ppc["branch"][CC_Branch_Index, :]


                if ppc_island["bus"].shape[0]!=0 and ppc_island["gen"].shape[0]!=0 and ppc_island["branch"].shape[0]!=0:
                    # if this is a meaningful island!
                    #print("Useful Island")
                    result_island = rundcpf_my(ppc_island)

                    # print(result_island[0]["bus"][:, 7]) # voltage mag, useless in DC power-flow!
                    # print(result_island[0]["bus"][:, 8])   # voltage angles!
                    voltageAngles[CC_Index[:, 0]] = result_island[0]["bus"][:, 8]

                    temp = ppc_island["branch"][:, 13]
                    temp = np.expand_dims(temp, axis = 1)
                    result_island[0]["branch"] = np.concatenate([result_island[0]["branch"], temp], axis = 1)

                    # print(result_island[0]["bus"].shape)  

                    result_tmp[0]["bus"] = np.vstack([result_tmp[0]["bus"], result_island[0]["bus"]]) if result_tmp[0]["bus"].size else result_island[0]["bus"]
                    result_tmp[0]["gen"] = np.vstack([result_tmp[0]["gen"], result_island[0]["gen"]]) if result_tmp[0]["gen"].size else result_island[0]["gen"]                                                  
                    result_tmp[0]["branch"] = np.vstack([result_tmp[0]["branch"], result_island[0]["branch"]]) if result_tmp[0]["branch"].size else result_island[0]["branch"]
                else:
                    # if this is NOT a meaningful island!
                    #print("Not a useful Island")
                    ppc_island["gen"][:, 1] = 0
                    ppc_island["bus"][:, 2] = 0

                    voltageAngles[CC_Index[:, 0]] = np.array([0]*len(CC_Index[:, 0]))

                    result_tmp[0]["bus"] = np.vstack([result_tmp[0]["bus"], ppc_island["bus"]]) if result_tmp[0]["bus"].size else ppc_island["bus"]
                    result_tmp[0]["gen"] = np.vstack([result_tmp[0]["gen"], ppc_island["gen"]]) if result_tmp[0]["gen"].size else ppc_island["gen"]  

                    temp1 = ppc_island["branch"][:, 0:ppc_island["branch"].shape[1]-1]
                    # temp1 = np.reshape(temp1, (-1, 1))
                    temp2 = np.zeros((ppc_island["branch"].shape[0], 4))
                    # temp2 = np.reshape(temp2, (-1, 1))
                    temp3 = ppc_island["branch"][:, ppc_island["branch"].shape[1]-1]
                    temp3 = np.reshape(temp3, (-1, 1))
                    #print(temp1, temp1.shape, temp2, temp2.shape, temp3, temp3.shape)
                    temp = np.concatenate((temp1, temp2, temp3), axis = 1)
                    #print(temp, temp.shape)
                    result_tmp[0]["branch"] = np.vstack([result_tmp[0]["branch"], temp]) if result_tmp[0]["branch"].size else temp
                    #print(result_tmp[0]["branch"].shape)  

            result[0]["bus"] = result_tmp[0]["bus"][result_tmp[0]["bus"][:, 0].argsort()]
            result[0]["gen"] = result_tmp[0]["gen"][result_tmp[0]["gen"][:, 0].argsort()]
            result[0]["branch"] = result_tmp[0]["branch"][result_tmp[0]["branch"][:, 17].argsort()]                                     
        else:
            result = rundcpf_my(ppc) 
            voltageAngles[:] = result[0]["bus"][:, 8]

            temp = ppc["branch"][:, 13]
            temp = np.expand_dims(temp, axis = 1)
            result[0]["branch"] = np.concatenate([result[0]["branch"], temp], axis = 1)
            result[0]["branch"] = result[0]["branch"][result[0]["branch"][:, 17].argsort()]

        #print("Completed performing power-flows for each island!")
        # print(result[0]["bus"][:, [0, 1, 2, 3]])
        # print(result[0]["branch"][:, [0, 1, 13]])

        for index in range(result[0]["branch"].shape[0]): 
            if np.isnan(result[0]["branch"][index, 13]):
                result[0]["branch"][index, 13] = 0
                result[0]["branch"][index, 15] = 0

        # print(tempur.shape)     

        tempur = np.multiply(tempur, adjacency)

        Node_index = np.zeros((adjacency.shape[0], 1))
        for index in range(adjacency.shape[0]):
            Node_index[index, 0] = index
           
        # print(result[0]["branch"].shape[0])
        # print(np.sum(link_flag_vector))
        # print(ppc["branch"].shape[0])
        ####################################################################################################
        # Checking Overloads
        ####################################################################################################

        #print("Checking for overloads................................")
        flow_vector = np.absolute(result[0]["branch"][:, 13])

        powerFlow = np.zeros((ppc["bus"].shape[0], ppc["bus"].shape[0]))
        for index in range(flow_vector.shape[0]):
            powerFlow[ int(result[0]["branch"][index, 0])-1, int(result[0]["branch"][index, 1])-1 ] = flow_vector[index]
            powerFlow[ int(result[0]["branch"][index, 1])-1, int(result[0]["branch"][index, 0])-1 ] = flow_vector[index]        

        # tempur = alpha*np.absolute( powerFlow ) + (1 - alpha)*np.absolute( tempur )
        tempur = alpha*np.absolute( powerFlow )

        # print((np.absolute( powerFlow ) > capacity).shape)
        # print(np.multiply(np.absolute( powerFlow ) > capacity, sparse.csr_matrix.todense(adjacency)))

        overload = np.max(np.multiply(np.absolute( powerFlow ) > capacity, sparse.csr_matrix.todense(adjacency)))
        # print(overload)
        
        # basically remove overloads and modify the adjacency matrix!
        adjacency = np.multiply(tempur <= capacity, sparse.csr_matrix.todense(adjacency))
   
        # use this to check for overloads and remove the appropriate lines from ppc!
        relevant_indices = np.argwhere(link_flag_vector == 1)
        # print(capacity_vector[relevant_indices[:,0]])
        # print(flow_vector.shape, capacity_vector.shape, capacity_vector[relevant_indices[:,0]].shape, np.squeeze(capacity_vector[relevant_indices[:,0]]).shape)

        condition = flow_vector > np.squeeze(capacity_vector[relevant_indices[:,0]])
        # print(condition)

        fail_index = np.argwhere( condition == True )
        fail_index = fail_index.astype(int)

        if fail_index.shape[0]==0:
            #print("No overloads! The system is normal after removing the intended lines.") 
            gen_shed = start_gen - np.sum(result[0]["gen"][:, 1])
            load_shed = start_load - np.sum(result[0]["bus"][:, 2])
            
        else:
            print("Overload detected! Starting the fast process............")
            overload = 1
            while overload:
                flow_vector = np.absolute(result[0]["branch"][:, 13])

                powerFlow = np.zeros((ppc["bus"].shape[0], ppc["bus"].shape[0]))
                for index in range(flow_vector.shape[0]):
                    powerFlow[ int(result[0]["branch"][index, 0])-1, int(result[0]["branch"][index, 1])-1 ] = flow_vector[index]
                    powerFlow[ int(result[0]["branch"][index, 1])-1, int(result[0]["branch"][index, 0])-1 ] = flow_vector[index]        

                # tempur = alpha*np.absolute( powerFlow ) + (1 - alpha)*np.absolute( tempur )
                tempur = alpha*np.absolute( powerFlow )

                # print((np.absolute( powerFlow ) > capacity).shape)
                # print(np.multiply(np.absolute( powerFlow ) > capacity, sparse.csr_matrix.todense(adjacency)))
                overload = np.max(np.multiply(np.absolute( powerFlow ) > capacity, adjacency))
                
                # print(overload)
                adjacency = np.multiply(tempur <= capacity, adjacency)
                relevant_indices = np.argwhere(link_flag_vector == 1)

                # print(capacity_vector[relevant_indices[:,0]])
                # print(flow_vector.shape, capacity_vector.shape, capacity_vector[relevant_indices[:,0]].shape, np.squeeze(capacity_vector[relevant_indices[:,0]]).shape)

                condition = flow_vector > np.squeeze(capacity_vector[relevant_indices[:,0]])
                # print(condition)

                fail_index = np.argwhere( condition == True )
                fail_index = fail_index.astype(int)

                # find the true index associated with the failed transmission line!
                tmp_index = ppc["branch"][fail_index, 13]
                tmp_index = tmp_index.astype(int)

                # all_outages = np.append(all_outages, tmp_index)
                # print("------")
                # print(fail_index.shape)
                # print(fail_index)
                # print(tmp_index.shape)
                # print(tmp_index)
                # print("------")

                if tmp_index.shape[0]>0:
                    link_flag_vector[tmp_index, 0] = 0

                for t in range(tmp_index.shape[0]):
                    ppc["branch"] = np.delete(ppc["branch"], (np.argwhere(ppc["branch"][:, 13] == tmp_index[t])), axis = 0)

                link_row = ppc["branch"][:, 0]                      # branch FROM
                link_row = np.expand_dims(link_row, axis=1)
                link_column = ppc["branch"][:, 1]                   # branch TO
                link_column = np.expand_dims(link_column, axis=1)

                # BFS traversal for connected components! Python not detecting single node islands! 
                # Temp fix in the lower half!
                ################################################################################################################
                # print(link_row, link_column)
                link_row = link_row.astype(int)
                link_column = link_column.astype(int)
                S, C = conncomp(link_row, link_column)
                ################################################################################################################

                # Load Shedding
                if S > 1:
                    result_tmp = copy.deepcopy(result)
                    result_tmp[0]["bus"] = np.array([])
                    result_tmp[0]["gen"] = np.array([])
                    result_tmp[0]["branch"] = np.array([])

                    for index in range(1, S+1):
                        CC_Index = np.argwhere(C == index)      # (-, 2) 2-D indices of numpy array satisfying given condition
                        #print(CC_Index[:, 0])
                        CC_Load = ppc["bus"][CC_Index[:, 0], 2] # (-,  )
                        CC_Gen = np.array([])
                        CC_Gen_Index = np.array([])             # (-,  )

                        # finding generators and generators indices in this island!
                        for j in range(ppc["gen"][:, 0].shape[0]):
                            if np.argwhere(CC_Index[:, 0] == ppc["gen"][j, 0]).shape[0]==1: 
                                CC_Gen = np.append(CC_Gen, ppc["gen"][j, 1])
                                CC_Gen_Index =  np.append(CC_Gen_Index, j)

                        CC_Gen_Index = CC_Gen_Index.astype(int)


                        """
                        # uncontrolled load-generation balance in each island via shedding!
                        if np.sum(CC_Load) > np.sum(CC_Gen):
                            if np.sum(CC_Gen) > 0:
                                CC_Load = CC_Load*(np.sum(CC_Gen)/np.sum(CC_Load))
                                #CC_Load = np.reshape(CC_Load, (-1, 1))
                                ppc["bus"][CC_Index[:, 0], 2] = CC_Load
                            else:    
                                ppc["gen"][CC_Gen_Index, 1] = 0
                                ppc["bus"][CC_Index[:, 0], 2] = 0
                        else:
                            if np.sum(CC_Load) < np.sum(CC_Gen):
                                if np.sum(CC_Load)>0:
                                    CC_Gen = CC_Gen*(np.sum(CC_Load)/np.sum(CC_Gen))
                                    # print(ppc["gen"][CC_Gen_Index, 1].shape, CC_Gen.shape)
                                    # CC_Gen = np.reshape(CC_Gen, (-1, 1))
                                    # print(ppc["gen"][CC_Gen_Index, 1].shape, CC_Gen.shape)
                                    ppc["gen"][CC_Gen_Index, 1] = CC_Gen
                                else:
                                    ppc["gen"][CC_Gen_Index, 1] = 0
                                    ppc["bus"][CC_Index[:, 0], 2] = 0
                        """    
                        
                        # controlled load-generation balance in each island!
                        if np.sum(CC_Load) < np.sum(CC_Gen):
                            if np.sum(CC_Load) > 0:
                                CC_Gen = CC_Gen*(np.sum(CC_Load)/np.sum(CC_Gen))
                                #CC_Gen = np.reshape(CC_Gen, (-1, 1))
                                ppc["gen"][CC_Gen_Index, 1] = CC_Gen
                            else:                
                                ppc["gen"][CC_Gen_Index, 1] = 0
                                ppc["bus"][CC_Index[:, 0], 2] = 0
                        elif np.sum(CC_Load) > np.sum(CC_Gen):
                            islandCapacity = np.sum(ppc["gen"][CC_Gen_Index, 8])
                            if np.sum(CC_Gen)==0:
                                ppc["gen"][CC_Gen_Index, 1] = 0
                                ppc["bus"][CC_Index[:, 0], 2] = 0
                            elif islandCapacity >= np.sum(CC_Load):
                                CC_Gen = CC_Gen + ppc["gen"][CC_Gen_Index, 8]*(np.sum(CC_Load) - np.sum(CC_Gen))/islandCapacity
                                #CC_Gen = np.reshape(CC_Gen, (-1, 1))
                                ppc["gen"][CC_Gen_Index, 1] = CC_Gen
                            else:
                                #print(np.sum(CC_Load), np.sum(CC_Gen), islandCapacity)
                                CC_Gen = ppc["gen"][CC_Gen_Index, 8]
                                #CC_Gen = np.reshape(CC_Gen, (-1, 1))
                                ppc["gen"][CC_Gen_Index, 1] = CC_Gen

                                CC_Load = CC_Load*(np.sum(CC_Gen)/np.sum(CC_Load))
                                #print(np.sum(CC_Load)) 
                                #CC_Load = np.reshape(CC_Load, (-1, 1))
                                ppc["bus"][CC_Index[:, 0], 2] = CC_Load
                                #print(np.sum(ppc["bus"][CC_Index[:, 0], 2]))

                        # print(np.sum(CC_Load), np.sum(CC_Gen))


                        ppc_island = copy.deepcopy(ppc)
                        ppc_island["bus"] = ppc["bus"][CC_Index[:, 0], :]
                        ppc_island["gen"] = ppc["gen"][CC_Gen_Index, :]
                        ppc_island["gencost"] = ppc["gencost"][CC_Gen_Index, :]

                        # print(np.sum(ppc_island["bus"][:, 2]), np.sum(ppc_island["gen"][:, 1]))

                        # assigning slack bus to the island if original not present to successfully solve power-flow! 

                        if np.argwhere(ppc_island["bus"][:, 1]==3).shape[0]==0 and ppc_island["bus"][:, 1].shape[0]>1:
                            island_slack_set = np.argwhere(ppc_island["bus"][:, 2] == min(ppc_island["bus"][:, 2]))  
                            #print(island_slack_set.shape)
                            island_slack = island_slack_set[0].item()
                            ppc_island["bus"][island_slack, 1] = 3


                        # print(ppc_island["bus"][:, 1]) 

                        # find all branches that belong to this island!
                        CC_Branch_Index = np.array([])
                        for t in range(ppc["branch"].shape[0]):
                            if np.argwhere(ppc_island["bus"][:, 0]==ppc["branch"][t, 0]).shape[0]>0 and np.argwhere(ppc_island["bus"][:, 0]==ppc["branch"][t, 1]).shape[0]>0: 
                                CC_Branch_Index = np.append(CC_Branch_Index, t)

                        CC_Branch_Index = CC_Branch_Index.astype(int)
                        ppc_island["branch"] = ppc["branch"][CC_Branch_Index, :]

                        if ppc_island["bus"].shape[0]!=0 and ppc_island["gen"].shape[0]!=0 and ppc_island["branch"].shape[0]!=0:
                            # if this is a meaningful island!!
                            # print(ppc_island["branch"].shape)
                            # print(ppc_island["bus"].shape)
                            # print(ppc_island["gen"])

                            result_island = rundcpf_my(ppc_island)   
                            voltageAngles[CC_Index[:, 0]] = result_island[0]["bus"][:, 8]
                            # print(result_island[0]["success"])

                            temp = ppc_island["branch"][:, 13]
                            temp = np.expand_dims(temp, axis = 1)
                            result_island[0]["branch"] = np.concatenate([result_island[0]["branch"], temp], axis = 1)

                            # print(result_island[0]["bus"].shape)  

                            result_tmp[0]["bus"] = np.vstack([result_tmp[0]["bus"], result_island[0]["bus"]]) if result_tmp[0]["bus"].size else result_island[0]["bus"]
                            result_tmp[0]["gen"] = np.vstack([result_tmp[0]["gen"], result_island[0]["gen"]]) if result_tmp[0]["gen"].size else result_island[0]["gen"]                                                  
                            result_tmp[0]["branch"] = np.vstack([result_tmp[0]["branch"], result_island[0]["branch"]]) if result_tmp[0]["branch"].size else result_island[0]["branch"]
                        else:
                            # if this is NOT a meaningful island!
                            ppc_island["gen"][:, 1] = 0
                            ppc_island["bus"][:, 2] = 0

                            voltageAngles[CC_Index[:, 0]] = np.array([0]*len(CC_Index[:, 0]))

                            result_tmp[0]["bus"] = np.vstack([result_tmp[0]["bus"], ppc_island["bus"]]) if result_tmp[0]["bus"].size else ppc_island["bus"]
                            result_tmp[0]["gen"] = np.vstack([result_tmp[0]["gen"], ppc_island["gen"]]) if result_tmp[0]["gen"].size else ppc_island["gen"]  

                            temp1 = ppc_island["branch"][:, 0:ppc_island["branch"].shape[1]-1]
                            # temp1 = np.reshape(temp1, (-1, 1))
                            temp2 = np.zeros((ppc_island["branch"].shape[0], 4))
                            # temp2 = np.reshape(temp2, (-1, 1))
                            temp3 = ppc_island["branch"][:, ppc_island["branch"].shape[1]-1]
                            temp3 = np.reshape(temp3, (-1, 1))
                            # print(temp1, temp1.shape, temp2, temp2.shape, temp3, temp3.shape)
                            temp = np.concatenate((temp1, temp2, temp3), axis = 1)
                            # print(temp, temp.shape)
                            result_tmp[0]["branch"] = np.vstack([result_tmp[0]["branch"], temp]) if result_tmp[0]["branch"].size else temp
                            # print(result_tmp[0]["branch"].shape)  

                    result[0]["bus"] = result_tmp[0]["bus"][result_tmp[0]["bus"][:, 0].argsort()]
                    result[0]["gen"] = result_tmp[0]["gen"][result_tmp[0]["gen"][:, 0].argsort()]
                    result[0]["branch"] = result_tmp[0]["branch"][result_tmp[0]["branch"][:, 17].argsort()]                                     
                else:
                    result = rundcpf_my(ppc)        

                    voltageAngles[:] = result[0]["bus"][:, 8]

                    temp = ppc["branch"][:, 13]
                    temp = np.expand_dims(temp, axis = 1)
                    result[0]["branch"] = np.concatenate([result[0]["branch"], temp], axis = 1)
                    result[0]["branch"] = result[0]["branch"][result[0]["branch"][:, 17].argsort()]

                for index in range(result[0]["branch"].shape[0]): 
                    if np.isnan(result[0]["branch"][index, 13]):
                        result[0]["branch"][index, 13] = 0
                        result[0]["branch"][index, 15] = 0

                tempur = np.multiply(tempur, adjacency)

                gen_shed = start_gen - np.sum(result[0]["gen"][:, 1])
                load_shed = start_load - np.sum(result[0]["bus"][:, 2])

        #print(result[0]["branch"].shape)
        #print(np.round(result[0]["branch"][:, [13, 15, 17]], 2))
        powerFlowVector = result[0]["branch"][:, 13]
        return (powerFlowVector, ppc, adjacency, S, gen_shed, load_shed, link_flag_vector)
####--------####--------####--------####--------####--------#####


    powerInjected_ori = powerInj_vector    
    powerInjected_ori = np.expand_dims(powerInjected_ori, axis=1)
    powerInjected_ori = np.transpose(powerInjected_ori)  # 1 x bus[0]
    adjacency_ori = sparse.csr_matrix(Adj)               # sparse adj
    reactance = sparse.csr_matrix(Reactance)             # sparse reactance

    M = np.argwhere(np.triu(sparse.csr_matrix.todense(adjacency_ori))!=0).shape[0]  # non-zero elements of the matrix

    link_row = branch[:, 0]                      # branch FROM
    link_row = np.expand_dims(link_row, axis=1)

    link_column = branch[:, 1]                   # branch TO
    link_column = np.expand_dims(link_column, axis=1)

    # perform this operation in order to identify the true line outage index!
    index_vector = np.expand_dims(np.sort(np.random.permutation(M)), axis = 1)  # 0 -- L-1
    ppc["branch"] = np.concatenate([ppc["branch"], index_vector], axis = 1)

    result = rundcpf_my(ppc)
    temp = ppc["branch"][:, 13]
    temp = np.expand_dims(temp, axis = 1)
    result[0]["branch"] = np.concatenate([result[0]["branch"], temp], axis = 1)

    alpha = 1
    K = 1
    link_set = np.concatenate([link_row, link_column], axis=1) 
    ##############################################
    # action to be selected --> ( 0 -- L-1 )
    # actionSpace = [14, 5]     # lines to outage! 
    voltageAngles = np.zeros((39,), dtype = float)
    ################################################

    FaultChains = np.zeros((K, 13))
    if len(actionSpace)==0:
        print(" ")
        print("############################### len(actionSpace) == 0 ###############################")
        print(" ")
        
        totalLoadShed = 0
        
        # Adj is numpy array, no issues!
        voltageAngles[:] = result[0]["bus"][:, 8]
        powerFlowVector = result[0]["branch"][:, 13]
        # [indices, capcacity, powerflow]
        actionDecision = np.c_[ppc["branch"][:, 13], ppc["branch"][:, 5], powerFlowVector]
        #print("----")
        #print("All in one: ", actionDecision)
        print(ppc["branch"].shape)
        #print("----")
        return (np.array(Adj, dtype = np.float32), voltageAngles, 0, False, totalLoadShed, actionDecision)
 
    elif len(actionSpace)==1:
        print(" ")
        print("############################### len(actionSpace) == 1 ###############################")
        print(" ")

        totalLoadShed = 0

        tmp_index = np.array([actionSpace[0]])   # these are line indices 0 -- (L-1) not 1 -- L !!
        initFail = link_set[tmp_index, :]        # failure indices
        powerFlowVector, ppcFirstOutage, adjacencyFirstOutage, S, gen_shed, load_shed, link_flag_vector = b_Pypower_DC_OPA_slack(voltageAngles, copy.deepcopy(ppc), adjacency_ori, Capacity, Capacity_vector, alpha, initFail, tmp_index)
        
        # [indices, capcacity, powerflow]
        actionDecision = np.c_[ppcFirstOutage["branch"][:, 13], ppcFirstOutage["branch"][:, 5], powerFlowVector]
        #print("----")
        #print("Line Indices: ", ppcFirstOutage["branch"][:, 13])
        #print("All in one: ", actionDecision)
        
        print(ppcFirstOutage["branch"].shape)
        #print("We have ", np.sum(link_flag_vector), " branches.")
        #print("We have ", S, " islands.")
        # print(ppcFirstOutage["branch"].shape)
        FaultChains[0, 0] = S
        FaultChains[0, 1] = gen_shed
        FaultChains[0, 2] = load_shed
        totalLoadShed += FaultChains[0, 2]
        FaultChains[0, 9] = totalLoadShed

        dataType = str(type(adjacencyFirstOutage))
        if "scipy.sparse" in dataType: 
            returnNumpyArray = np.array(adjacencyFirstOutage.toarray(), dtype = np.float32)
        else:
            returnNumpyArray = np.array(adjacencyFirstOutage, dtype = np.float32)
        
        # this is supposed to return the load_shed in the current round!
        #print("----")
        return (returnNumpyArray.reshape((39, 39)), voltageAngles, load_shed, False, totalLoadShed, actionDecision)
 
    elif len(actionSpace)==2:
        print(" ")
        print("############################### len(actionSpace) == 2 ###############################")
        print(" ")

        totalLoadShed = 0

        tmp_index = np.array([actionSpace[0]])   # these are line indices 0 -- (L-1) not 1 -- L !!
        initFail = link_set[tmp_index, :]        # failure indices
        powerFlowVector, ppcFirstOutage, adjacencyFirstOutage, S, gen_shed, load_shed, link_flag_vector = b_Pypower_DC_OPA_slack(voltageAngles, copy.deepcopy(ppc), adjacency_ori, Capacity, Capacity_vector, alpha, initFail, tmp_index)
        
        # [indices, capcacity, powerflow]
        #actionDecision = np.c_[ppcFirstOutage["branch"][:, 13], ppcFirstOutage["branch"][:, 5], powerFlowVector]
        #print("----")
        #print("Line Indices: ", ppcFirstOutage["branch"][:, 13]) 
        #print("All in one: ", actionDecision)
        
        print(ppcFirstOutage["branch"].shape)
        #print("We have ", np.sum(link_flag_vector), " branches.")
        #print("We have ", S, " islands.")
        # print(ppcFirstOutage["branch"].shape)
        FaultChains[0, 0] = S
        FaultChains[0, 1] = gen_shed
        FaultChains[0, 2] = load_shed
        totalLoadShed += FaultChains[0, 2]
        FaultChains[0, 9] = totalLoadShed
        

        sp_adjacencyFirstOutage = sparse.csr_matrix(adjacencyFirstOutage)              # sparse adj
        tmp_index = np.array([actionSpace[1]])   # these are line indices 0 -- (L-1) not 1 -- L !!
        initFail = link_set[tmp_index, :]        # failure indices
        powerFlowVector, ppcSecondOutage, adjacencySecondOutage, S, gen_shed, load_shed, link_flag_vector = b_Pypower_DC_OPA_slack(voltageAngles, copy.deepcopy(ppcFirstOutage), sp_adjacencyFirstOutage, Capacity, Capacity_vector, alpha, initFail, tmp_index, link_flag_vector)
        
        # [indices, capcacity, powerflow]
        actionDecision = np.c_[ppcSecondOutage["branch"][:, 13], ppcSecondOutage["branch"][:, 5], powerFlowVector]        
        #print("----")
        #print("Line Indices: ", ppcSecondOutage["branch"][:, 13])
        #print("All in one: ", actionDecision)
        
        print(ppcSecondOutage["branch"].shape)
        #print("We have ", np.sum(link_flag_vector), " branches.")
        #print("We have ", S, " islands.")
        #print(voltageAngles)
        # print(ppcFirstOutage["branch"].shape)
        FaultChains[0, 3] = S
        FaultChains[0, 4] = gen_shed
        FaultChains[0, 5] = load_shed
        totalLoadShed += FaultChains[0, 5]
        FaultChains[0, 9] = totalLoadShed
        
        dataType = str(type(adjacencySecondOutage))
        if "scipy.sparse" in dataType: 
            returnNumpyArray = np.array(adjacencySecondOutage.toarray(), dtype = np.float32)
        else:
            returnNumpyArray = np.array(adjacencySecondOutage, dtype = np.float32)
        
        #print("----")
        return (returnNumpyArray.reshape((39, 39)), voltageAngles, load_shed, False, totalLoadShed, actionDecision)     

    
    elif len(actionSpace)==3: 
        print(" ")
        print("############################### len(actionSpace) == 3 ###############################")
        print(" ")

        totalLoadShed = 0

        tmp_index = np.array([actionSpace[0]])   # these are line indices 0 -- (L-1) not 1 -- L !!
        initFail = link_set[tmp_index, :]        # failure indices
        powerFlowVector, ppcFirstOutage, adjacencyFirstOutage, S, gen_shed, load_shed, link_flag_vector = b_Pypower_DC_OPA_slack(voltageAngles, copy.deepcopy(ppc), adjacency_ori, Capacity, Capacity_vector, alpha, initFail, tmp_index)
        
        # [indices, capcacity, powerflow]
        #actionDecision = np.c_[ppcFirstOutage["branch"][:, 13], ppcFirstOutage["branch"][:, 5], powerFlowVector]
        #print("----")
        #print("Power Flow in Line: ", powerFlowVector)
        #print("Line Capacity: ", ppcFirstOutage["branch"][:, 5])
        #print("Line Indices: ", ppcFirstOutage["branch"][:, 13])
        #print("All in one: ", actionDecision)
        
        print(ppcFirstOutage["branch"].shape)
        #print("We have ", np.sum(link_flag_vector), " branches.")
        #print("We have ", S, " islands.")
        FaultChains[0, 0] = S
        FaultChains[0, 1] = gen_shed
        FaultChains[0, 2] = load_shed
        totalLoadShed += FaultChains[0, 2]
        FaultChains[0, 9] = totalLoadShed
        
        
        sp_adjacencyFirstOutage = sparse.csr_matrix(adjacencyFirstOutage)              # sparse adj
        tmp_index = np.array([actionSpace[1]])   # these are line indices 0 -- (L-1) not 1 -- L !!
        initFail = link_set[tmp_index, :]        # failure indices
        powerFlowVector, ppcSecondOutage, adjacencySecondOutage, S, gen_shed, load_shed, link_flag_vector = b_Pypower_DC_OPA_slack(voltageAngles, copy.deepcopy(ppcFirstOutage), sp_adjacencyFirstOutage, Capacity, Capacity_vector, alpha, initFail, tmp_index, link_flag_vector)
        
        # [indices, capcacity, powerflow]
        #actionDecision = np.c_[ppcSecondOutage["branch"][:, 13], ppcSecondOutage["branch"][:, 5], powerFlowVector]
        #print("----")
        #print("Power Flow in Line: ", powerFlowVector)
        #print("Line Capacity: ", ppcSecondOutage["branch"][:, 5])
        #print("Line Indices: ", ppcSecondOutage["branch"][:, 13])
        #print("All in one: ", actionDecision)
        print(ppcSecondOutage["branch"].shape)
        #print("We have ", np.sum(link_flag_vector), " branches.")
        #print("We have ", S, " islands.")
        #print(voltageAngles)
        #print(ppcFirstOutage["branch"].shape)
        FaultChains[0, 3] = S
        FaultChains[0, 4] = gen_shed
        FaultChains[0, 5] = load_shed
        totalLoadShed += FaultChains[0, 5]
        FaultChains[0, 9] = totalLoadShed    


        sp_adjacencySecondOutage = sparse.csr_matrix(adjacencySecondOutage)              # sparse adj
        tmp_index = np.array([actionSpace[2]])   # these are line indices 0 -- (L-1) not 1 -- L !!
        initFail = link_set[tmp_index, :]        # failure indices
        powerFlowVector, ppcThirdOutage, adjacencyThirdOutage, S, gen_shed, load_shed, link_flag_vector = b_Pypower_DC_OPA_slack(voltageAngles, copy.deepcopy(ppcSecondOutage), sp_adjacencySecondOutage, Capacity, Capacity_vector, alpha, initFail, tmp_index, link_flag_vector)
        
        # [indices, capcacity, powerflow]
        actionDecision = np.c_[ppcThirdOutage["branch"][:, 13], ppcThirdOutage["branch"][:, 5], powerFlowVector]
        #print("----")
        #print("Power Flow in Line: ", powerFlowVector)
        #print("Line Capacity: ", ppcThirdOutage["branch"][:, 5])
        #print("Line Indices: ", ppcThirdOutage["branch"][:, 13])
        #print("All in one: ", actionDecision)
        print(ppcThirdOutage["branch"].shape)
        #print("We have ", np.sum(link_flag_vector), " branches.")
        #print("We have ", S, " islands.")
        #print(voltageAngles)
        #print(ppcFirstOutage["branch"].shape)
        FaultChains[0, 6] = S
        FaultChains[0, 7] = gen_shed
        FaultChains[0, 8] = load_shed
        totalLoadShed += FaultChains[0, 8]

        dataType = str(type(adjacencyThirdOutage))
        if "scipy.sparse" in dataType: 
            returnNumpyArray = np.array(adjacencyThirdOutage.toarray(), dtype = np.float32)
        else:
            returnNumpyArray = np.array(adjacencyThirdOutage, dtype = np.float32)
        
        #print("----")
        return (returnNumpyArray.reshape((39, 39)), voltageAngles, load_shed, True, totalLoadShed, actionDecision)     
    

    FaultChains[0, 9] = totalLoadShed
    FaultChains[0, [10, 11, 12]] = actionSpace
        

In [ ]:
#  just to check if everything works okay!
actionSpace = [6, 28, 37]
currentAnswer = environmentStep(actionSpace, 0.5)
#print(currentAnswer[0], currentAnswer[1], currentAnswer[2], currentAnswer[3], currentAnswer[4])
#print(currentAnswer[4])

In [ ]:
# N-3 data generation and storage!
start = time.time()

#upperLimit = 15
loadingCondition = 0.7
limit, currRow = 46, 0
data_to_write = np.zeros((limit*(limit-1)*(limit-2), 4), dtype=int)
for iter1 in range(limit):
    for iter2 in range(iter1+1, limit):
        for iter3 in range(iter2+1, limit):
            currPermutation = list(itertools.permutations((iter1, iter2, iter3)))
            for eachTuple in currPermutation: 
                data_to_write[currRow, 0] = eachTuple[0]
                data_to_write[currRow, 1] = eachTuple[1]
                data_to_write[currRow, 2] = eachTuple[2]
                actionSpace = list(eachTuple)
                _, _, _, _, totalreward = environmentStep(actionSpace, loadingCondition)
                data_to_write[currRow, 3] = totalreward
                currRow += 1            
            
#print(len(data_to_write))
#print(data_to_write)

# took 7570.848524808884 time  
#with h5py.File('loading07_39bus.h5', 'w') as hf:
#    hf.create_dataset("loading07_39bus",  data=data_to_write)  
  
end = time.time() 
print(end - start)

In [ ]:
with h5py.File('loading06_39bus.h5', 'r') as hf:
    dataSet = hf['loading06_39bus'][:]

print(dataSet)
print(len(dataSet))

In [ ]:
# analyzing the statistics of fault chains dataset!
riskyFaultChains = dataSet[dataSet[:, 3]>-1]
riskyFaultChainDict = {}
for index in range(riskyFaultChains.shape[0]):
    riskyFaultChainDict[tuple(riskyFaultChains[index, [0, 1, 2]])] = riskyFaultChains[index, 3]
print(len(riskyFaultChainDict))
riskyFaultChainDict